In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, expr, radians, sin, cos, sqrt, asin, lit, row_number, power
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [ ]:


class RestaurantRecommenderSystem:
    def __init__(self, spark_session):
        """
        Initialize the Restaurant Recommender System
        """
        self.spark = spark_session
        self.df = None
        self.kmeans_model = None
        self.scaler = None
        self.vector_assembler = None

    def load_and_preprocess_data(self, data_path):
        """
        Load and preprocess restaurant data
        """
        # Read the CSV file
        self.df = self.spark.read.csv('/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv', header=True, inferSchema=True)

        # Data cleaning and preprocessing
        self.df = self.df.na.drop()  # Remove rows with null values

        # Convert boolean columns
        self.df = (self.df.withColumn("has_parking",
                    expr("CASE WHEN parking = 'Yes' THEN true ELSE false END"))
                   .withColumn("has_wifi",
                    expr("CASE WHEN WiFi = 'Yes' THEN true ELSE false END")))

        # Encode categorical variables
        cuisine_indexer = StringIndexer(inputCol="cuisine_type", outputCol="cuisine_type_encoded")
        # Fit and transform the DataFrame using the StringIndexer
        self.df = cuisine_indexer.fit(self.df).transform(self.df)

        return self.df

    def create_feature_vector(self):
        """
        Create feature vector for clustering
        """
        # Select features for clustering
        feature_columns = [
            "latitude",
            "longitude",
            "rating",
            "review_count",
            "cuisine_type_encoded"
        ]

        # Vector Assembler to combine features
        self.vector_assembler = VectorAssembler(
            inputCols=feature_columns,
            outputCol="features"
        )

        # Scale features
        self.scaler = StandardScaler(
            inputCol="features",
            outputCol="scaled_features",
            withStd=True,
            withMean=True
        )

        return self

    def train_kmeans_clustering(self, k=5):
        """
        Train K-means clustering model
        """
        # Create K-means model
        kmeans = KMeans(
            featuresCol="scaled_features",
            predictionCol="cluster",
            k=k
        )

        # Create pipeline
        pipeline = Pipeline(stages=[
            self.vector_assembler,
            self.scaler,
            kmeans
        ])

        # Fit the pipeline
        self.kmeans_model = pipeline.fit(self.df)

        return self

    def save_model(self, model_path):
        """
        Save the trained model
        """
        # Instead of joblib, use PySpark's save method for PipelineModels
        self.kmeans_model.save(os.path.join('/content/drive/MyDrive/Models', 'restaurant_recommender_Final_model'))
        print(f"Model saved to {model_path}")

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate distance between two geographical points
        """
        lat1_rad = radians(lit(lat1))
        lon1_rad = radians(lit(lon1))
        lat2_rad = radians(lat2)
        lon2_rad = radians(lon2)

        return (
            6371 * 2 * asin(
                sqrt(
                    power(sin((lat2_rad - lat1_rad) / 2), 2) +
                    cos(lat1_rad) * cos(lat2_rad) *
                    power(sin((lon2_rad - lon1_rad) / 2), 2)
                )
            )
        )

    def recommend_restaurants(
        self,
        user_location,
        min_rating,
        need_parking,
        need_wifi,
        cuisine_type,
        max_distance=60
    ):
        """
        Recommend restaurants based on user preferences
        """
        user_lat, user_lon = user_location

        # Apply initial filters
        filtered_df = self.df.filter(
            (col("rating") >= min_rating) &
            (col("cuisine_type") == cuisine_type)
        )

        # Apply parking filter
        if need_parking:
            filtered_df = filtered_df.filter(col("has_parking") == True)

        # Apply WiFi filter
        if need_wifi:
            filtered_df = filtered_df.filter(col("has_wifi") == True)

        # Add distance column
        with_distance_df = filtered_df.withColumn(
            "distance",
            self.haversine_distance(user_lat, user_lon, col("latitude"), col("longitude"))
        )

        # Filter by distance
        nearby_restaurants = with_distance_df.filter(
            col("distance") <= max_distance
        )

        # Rank restaurants
        window_spec = Window.partitionBy("bus_name").orderBy(col("rating").desc())

        recommended_restaurants = (
            nearby_restaurants
            .withColumn("rank", row_number().over(window_spec))
            .filter(col("rank") == 1)
            .orderBy(col("rating").desc())
            .select("business_id", "bus_name", "address", "rating", "distance", "cuisine_type")
            .limit(5)
        )

        return recommended_restaurants

def interactive_recommendation(recommender):
    """
    Interactive restaurant recommendation interface
    """
    print("Restaurant Recommender System")

    # Get user inputs
    user_lat = float(input("Enter your latitude: "))
    user_lon = float(input("Enter your longitude: "))
    min_rating = float(input("Enter minimum rating (0-5): "))
    need_parking = input("Need parking? (yes/no): ").lower() == 'yes'
    need_wifi = input("Need WiFi? (yes/no): ").lower() == 'yes'
    cuisine_type = input("Enter cuisine type: ")

    # Make recommendations
    recommendations = recommender.recommend_restaurants(
        user_location=(user_lat, user_lon),
        min_rating=min_rating,
        need_parking=need_parking,
        need_wifi=need_wifi,
        cuisine_type=cuisine_type
    )

    # Show recommendations
    recommendations.show()



In [ ]:
def main():
    # Create Spark Session
    spark = SparkSession.builder \
        .appName("RestaurantRecommenderSystem") \
        .getOrCreate()

    # Initialize recommender
    recommender = RestaurantRecommenderSystem(spark)

    # Define paths
    data_path = "/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv"
    model_save_path = "/content/drive/MyDrive/Models/restaurant_recommender_Final_model.pkl"

    # Load and preprocess data
    recommender.load_and_preprocess_data(data_path)

    # Create feature vector
    recommender.create_feature_vector()

    # Train K-means clustering
    recommender.train_kmeans_clustering(k=5)

    # Save model
    recommender.save_model(model_save_path)

    # Start interactive recommendation
    interactive_recommendation(recommender)

if __name__ == "__main__":
    main()

Model saved to /content/drive/MyDrive/Models/restaurant_recommender_Final_model.pkl
Restaurant Recommender System
Enter your latitude: 53.5187059314
Enter your longitude: -113.4975095574
Enter minimum rating (0-5): 3.5
Need parking? (yes/no): No
Need WiFi? (yes/no): No
Enter cuisine type: American
+--------------------+--------------------+--------------------+------+------------------+------------+
|         business_id|            bus_name|             address|rating|          distance|cuisine_type|
+--------------------+--------------------+--------------------+------+------------------+------------+
|tGl2jBbSVPEl27cw9...|Danny Hacksaw's P...|1503 Lakewood Road W|   4.5| 7.143981489706797|    American|
|sFjzcP5GXcCmQAGOG...|          Red Ox Inn|   9420 91 Street NW|   4.5| 2.455386840412777|    American|
|if2ZvtLUOdn6EFeWj...|     El Fogón Latino|     8026 118 Avenue|   4.5| 6.226928045014033|    American|
|G5jxVV_AIHCquF0B3...|            Ernest's|Hokanson Centre f...|   4.5| 5.704